# TROPOMI functions

In [ ]:
def TROPOMI_download(input_type, bbox, period, product_type, processing_mode):

   """ Query and download the TROPOMI dataset from Sentinel API

         Args:
            input_type (str): Search type (Manual or Query)
            bbox (arr): Query bounding box
            period (list): Query period of time 
            product_type (str): Query product type
            processing_mode (str): Query processing mode (Offline, Near real time, Reprocessing)

        Returns:
            file_name (str): File name of TROPOMI product within 5phub
            product_name (str): Product name of TROPOMI product within 5phub
   """

   user = 's5pguest' 
   password = 's5pguest' 
   api = SentinelAPI(user, password, 'https://s5phub.copernicus.eu/dhus/')

   if input_type == 'Manual':

      file_name = input('Write file name: ')
      product_name = input('Write product name:')

   elif input_type == 'Query':
      
      print('Warning: The maximum number of items that can be shown is 5.')
      print('You can see all the results at https://s5phub.copernicus.eu/dhus/.')

      poly = geojson.Polygon([[(bbox[0][0], bbox[0][1]), (bbox[0][0], bbox[1][1]), 
                               (bbox[1][0], bbox[1][1]), (bbox[1][0], bbox[0][1]), 
                               (bbox[0][0], bbox[0][1])]])

      products = api.query(area = geojson_to_wkt(poly),
                           area_relation = 'Contains',
                           producttype = product_type,
                           processinglevel = 'L2',
                           platformname = 'Sentinel-5 Precursor',
                           instrumentname = 'TROPOspheric Monitoring Instrument',
                           processingmode = processing_mode,
                           date = period,
                           limit = 5)

      items = list(products.items())
      
      if not items: 
         print('There are no results. The code will be interrupted.')
         raise KeyboardInterrupt
       
      else:
         print('RESULTS FOR PERIOD ' + str(period))
         for i in range(0, len(items)):
            print('Number ', i, ': ', items[i][1]['title'], sep = '')

      file_int = input('Select number or press Enter if you want to select the first result: ') or 0
      file_name = items[int(file_int)][0]
      product_name = items[int(file_int)][1]['title'] + '.nc'

   print('SELECTED')
   print('File name:', file_name)
   print('Product name:', product_name)
   
   if os.path.isfile(os.path.join(os.path.abspath(''), 'data/' + sensor + '/' + component_nom + '/' + product_name)):
      print('The file exists, it will not be downloaded again.')

   else:
      print('The file does not exist, it will be downloaded.')
      api.download(file_name, directory_path = 'data/tropomi/' + component_nom)

   return file_name, product_name

In [ ]:
def TROPOMI_read(product_names, component_nom):

    """ Read TROPOMI dataset as xarray dataset object

        Args:
            product_name (str): Product name of TROPOMI product within 5phub
            component_nom (str): Component chemical nomenclature

        Returns:
            TROPOMI_ds (xarray): TROPOMI dataset in xarray format
            support_input_ds (xarray): TROPOMI dataset that contains support input data in xarray format
            support_details_ds (xarray): TROPOMI dataset that contains support details data in xarray format
    """

    TROPOMI_ds_all = []
    support_input_ds_all = []
    support_details_ds_all = []

    for product_name in product_names:

        TROPOMI_ds = xr.open_dataset('data/tropomi/' + component_nom + '/' + product_name, group = 'PRODUCT')

        support_input_ds = xr.open_dataset('data/tropomi/' + component_nom + '/' + product_name, 
                                           group = 'PRODUCT/SUPPORT_DATA/INPUT_DATA')

        support_input_ds = support_input_ds.assign(ground_pixel = TROPOMI_ds.ground_pixel)
        support_input_ds = support_input_ds.assign(scanline = TROPOMI_ds.scanline)
        support_input_ds = support_input_ds.assign(time = TROPOMI_ds.time)
        support_input_ds = support_input_ds.set_coords(['ground_pixel', 'scanline', 'time'])

        support_details_ds = xr.open_dataset('data/tropomi/' + component_nom + '/' + product_name, 
                                             group = 'PRODUCT/SUPPORT_DATA/DETAILED_RESULTS')

        if component_nom == 'CO':
            
            # Transform heights into levels
            data = {'Layer': np.arange(1, 51)[::-1], 'Height': TROPOMI_ds.layer}
            dataframe = pd.DataFrame(data)
            
            for i in range(0, 50):

                TROPOMI_ds['layer'] = xr.where(TROPOMI_ds.layer == dataframe['Height'].iloc[i], 
                                            int(dataframe['Layer'].iloc[i]), TROPOMI_ds['layer'])

        TROPOMI_ds_all.append(TROPOMI_ds)
        support_input_ds_all.append(support_input_ds)
        support_details_ds_all.append(support_details_ds)

    # Merge if there is more than one product
    if len(product_names) >= 2:

        TROPOMI_ds = xr.merge(TROPOMI_ds_all)
        support_input_ds = xr.merge(support_input_ds_all)
        support_details_ds = xr.merge(support_input_ds_all)

    return TROPOMI_ds, support_input_ds, support_details_ds

In [ ]:
def TROPOMI_pressure(TROPOMI_ds, component_nom, support_input_ds, support_details_ds):

    """ Calculate level pressures for TROPOMI dataset

        Args:
            TROPOMI_ds (xarray): TROPOMI dataset in xarray format
            component_nom (str): Component chemical nomenclature
            support_input_ds (xarray): TROPOMI dataset that contains support input data in xarray format
            support_details_ds (xarray): TROPOMI dataset that contains support details data in xarray format
            
        Returns:
            TROPOMI_ds (xarray): TROPOMI dataset in xarray format
    """

    if component_nom == 'NO2':
        
        print('The layers pressures will be calculated (lower and upper bounds).')

        # Calculate pressure as p = ap + b * ps (Units: ap(Pa) + b(none) * ps(Pa) -> To Pa)
        pressure = (TROPOMI_ds.tm5_constant_a + TROPOMI_ds.tm5_constant_b * support_input_ds.surface_pressure)
        TROPOMI_ds = TROPOMI_ds.assign(pressure = pressure)
    
    elif component_nom == 'CO':

        print('The layers pressures will be retrieved (lower bound).')

        # Pressure is at lower bound!
        pressure_lower = support_details_ds.pressure_levels
        TROPOMI_ds = TROPOMI_ds.assign(pressure = pressure_lower)

    elif component_nom == 'SO2':
        
        print('The layers pressures will be calculated (unknown bound, half?).')
        
        pressure = (support_input_ds.tm5_constant_a + support_input_ds.tm5_constant_b * support_input_ds.surface_pressure)
        TROPOMI_ds = TROPOMI_ds.assign(pressure = pressure)
    
    elif component_nom == 'O3':

        print('The layers pressures will be retrieved (unknown bound, half?).')
        pressure = support_details_ds.pressure_grid
        TROPOMI_ds = TROPOMI_ds.assign(pressure = pressure)

    else:
        print('This dataset does not contain data to calculate the layer pressures.')

    return TROPOMI_ds

In [ ]:
def TROPOMI_kernel_column(TROPOMI_ds, support_details_ds):

    """ Calculate column kernels for TROPOMI dataset

        Args:
            TROPOMI_ds (xarray): TROPOMI dataset in xarray format
            support_details_ds (xarray): TROPOMI dataset that contains support details data in xarray format
        
        Returns:
            TROPOMI_ds (xarray): TROPOMI dataset in xarray format
    """

    if component_nom == 'NO2':

        kernel_column = xr.where(TROPOMI_ds.layer > TROPOMI_ds.tm5_tropopause_layer_index, 0, 
                                 TROPOMI_ds.averaging_kernel * (TROPOMI_ds.air_mass_factor_total / 
                                 TROPOMI_ds.air_mass_factor_troposphere))
        TROPOMI_ds = TROPOMI_ds.assign(kernel_column = kernel_column)

    elif component_nom == 'CO':

        kernel_column = support_details_ds.column_averaging_kernel
        TROPOMI_ds = TROPOMI_ds.assign(kernel_column = kernel_column)
    
    elif component_nom == 'SO2':
        
        height_options = [1, 7, 15]
        height = input('Input height (in km) to calculate the column kernels with accuracy: ')

        while int(height) not in height_options:
            print('ERROR: Enter a valid height number. The options are 1, 7 or 15 km.')
            height = input('Input height (in km): ')
        
        # WRONG: This formula must be corrected. Troposphere AMF do not appear in the dataset.
        kernel_column = support_details_ds.averaging_kernel * support_details_ds['sulfurdioxide_total_air_mass_factor_' 
                                                                                 + height + 'km']
        TROPOMI_ds = TROPOMI_ds.assign(kernel_column = kernel_column)
    
    else:
        print('The dataset does not contain information to retrieve or calculate the column averaging kernels.')

    return TROPOMI_ds

In [ ]:
def TROPOMI_profile_apriori(TROPOMI_ds, component, support_details_ds):

    """ Create file with the original corresponding coordinates to each scanline and ground pixel in TROPOMI dataset

        Args:
            TROPOMI_ds (xarray): TROPOMI dataset in xarray format
            component (str): Component name
            support_details_ds (xarray): TROPOMI dataset that contains support details data in xarray format
        
        Returns:
            TROPOMI_ds (xarray): TROPOMI dataset in xarray format
    """

    profile_apriori_name = component.replace('_', '') + '_profile_apriori'

    if profile_apriori_name in list(support_details_ds.keys()):
        profile_apriori = support_details_ds[profile_apriori_name]
        TROPOMI_ds = TROPOMI_ds.assign(profile_apriori = profile_apriori)

    else:
        print('The dataset does not contain any apriori profile.')
    
    return TROPOMI_ds

In [ ]:
def TROPOMI_original_coords(TROPOMI_ds, component_nom, time):

    """ Create file with the original corresponding coordinates to each scanline and ground pixel in TROPOMI dataset

        Args:
            TROPOMI_ds (xarray): TROPOMI dataset in xarray format
            component_nom (str): Component chemical nomenclature
            time (timestamp): Start datetime for each period
    """

    # Create dataframe with scanlines and ground pixels
    TROPOMI_coords_df = []
    TROPOMI_coords_df = pd.DataFrame(list(product(TROPOMI_ds.ground_pixel.values, TROPOMI_ds.scanline.values)), 
                            columns = ['ground_pixel', 'scanline'])

    # Find corresponding latitudes and longitudes                          
    for index, row in TROPOMI_coords_df.iterrows():
        TROPOMI_coords_df.loc[index,'latitude'] = TROPOMI_ds.latitude.sel(
                                                scanline = TROPOMI_coords_df['scanline'].loc[index], 
                                                ground_pixel = TROPOMI_coords_df['ground_pixel'].loc[index],
                                                method = None).values
                                            
        TROPOMI_coords_df.loc[index,'longitude'] = TROPOMI_ds.longitude.sel(
                                                scanline = TROPOMI_coords_df['scanline'].loc[index], 
                                                ground_pixel = TROPOMI_coords_df['ground_pixel'].loc[index],
                                                method = None).values

    # Save as csv
    TROPOMI_coords_df.to_csv('data/tropomi/' +  component_nom + '/' + component_nom + '-coords-' + str(time) + '.csv', index = False)

In [ ]:
def TROPOMI_subset(TROPOMI_ds, product_name, bbox, time):

    """ Read file with the corresponding coordinates to each scanline and ground pixel in TROPOMI dataset.
        Subset dataset into desired bounding box.

        Args:
            TROPOMI_ds (xarray): TROPOMI dataset in xarray format
            product_name (str): Product name of TROPOMI product within 5phub
            bbox (arr): Query bounding box
            time (timestamp): Start datetime for each period
    
        Returns:
            TROPOMI_ds (xarray): TROPOMI dataset in xarray format
    """

    if os.path.isfile(os.path.join(os.path.abspath(''), 'data/' + sensor + '/' +  component_nom + '/' + component_nom + '-coords-' + str(time) + '.csv')):
        pass

    else: 
        TROPOMI_original_coords(TROPOMI_ds, component_nom, time)
        
    # Read csv
    TROPOMI_coords_df = pd.read_csv('data/tropomi/' +  component_nom + '/' + component_nom + '-coords-' + str(time) + '.csv')
    
    # Set limits
    TROPOMI_coords_df = TROPOMI_coords_df[(TROPOMI_coords_df['latitude'] >= bbox[0][1]) & 
                                          (TROPOMI_coords_df['latitude'] <= bbox[1][1])]
    TROPOMI_coords_df = TROPOMI_coords_df[(TROPOMI_coords_df['longitude'] >= bbox[0][0]) & 
                                          (TROPOMI_coords_df['longitude'] <= bbox[1][0])]
    
    if TROPOMI_coords_df.empty:
        
        print('ERROR: The subset could not be made. Try to find a new TROPOMI dataset. The code will be interrupted.')
        raise KeyboardInterrupt

    else:
        
        # Get scanline and ground pixel coordinates
        scanline_coords = np.unique(TROPOMI_coords_df['scanline'].values).tolist()
        ground_pixel_coords = np.unique(TROPOMI_coords_df['ground_pixel'].values).tolist()

        # Set limits
        TROPOMI_ds = TROPOMI_ds.sel(scanline = scanline_coords, ground_pixel = ground_pixel_coords)

    return TROPOMI_ds

In [ ]:
def TROPOMI_subset_coords(TROPOMI_ds):

    """ Create file with the subset corresponding coordinates to each scanline and ground pixel in TROPOMI dataset

        Args:
            TROPOMI_ds (xarray): TROPOMI dataset in xarray format
        
        Returns:
            TROPOMI_subset_coords_df (dataframe): Dataframe with subset coordinates
    """

    # Create dataframe with scanlines and ground pixels
    TROPOMI_subset_coords_df = []
    TROPOMI_subset_coords_df = pd.DataFrame(list(product(TROPOMI_ds.ground_pixel.values, TROPOMI_ds.scanline.values)), 
                                            columns = ['ground_pixel', 'scanline'])

    # Find corresponding latitudes and longitudes                          
    for index, row in TROPOMI_subset_coords_df.iterrows():
        
        TROPOMI_subset_coords_df.loc[index,'latitude'] = TROPOMI_ds.latitude.sel(
                                                        scanline = TROPOMI_subset_coords_df['scanline'].loc[index], 
                                                        ground_pixel = TROPOMI_subset_coords_df['ground_pixel'].loc[index],
                                                        method = None).values
                                                    
        TROPOMI_subset_coords_df.loc[index,'longitude'] = TROPOMI_ds.longitude.sel(
                                                        scanline = TROPOMI_subset_coords_df['scanline'].loc[index], 
                                                        ground_pixel = TROPOMI_subset_coords_df['ground_pixel'].loc[index],
                                                        method = None).values

    return TROPOMI_subset_coords_df

In [ ]:
def TROPOMI_convert_units(TROPOMI_ds, sensor_column):

    """ Convert the units of the total column of TROPOMI dataset for any component from mol/m2 to molecules/cm2

        Args:
            TROPOMI_ds (xarray): TROPOMI dataset in xarray format
            sensor_column (str): Name of sensor column in downloaded dataset
        
        Returns:
            TROPOMI_ds (xarray): TROPOMI dataset in xarray format
    """

    TROPOMI_ds[sensor_column] = TROPOMI_ds[sensor_column] * 6.02214*10**19
    print('The units of the component columns have been converted to molecules/cm2.')
    
    if 'profile_apriori' in list(TROPOMI_ds.keys()):
        
        TROPOMI_ds['profile_apriori'] = TROPOMI_ds['profile_apriori'] * 6.02214*10**19
        print('The units of the apriori profiles have been converted to molecules/cm2.')

    return TROPOMI_ds

In [ ]:
def TROPOMI_prepare_df(match_df):

    """ Prepare dataframe for merge

        Args:
            match_df (dataframe): Dataframe used to apply averaging kernels
        
        Returns:
            match_df (dataframe): Dataframe used to apply averaging kernels
    """

    # Pass NaNs to data with qa_value under 0.5
    match_df.loc[match_df['qa_value'] < 0.5, [sensor_column, 'kernel_column']] = float('NaN')

    # Drop levels
    if component_nom == 'CO' or component_nom == 'SO2':
        
        match_df.index.names = ['corner', 'ground_pixel', 'layer', 'scanline']
    
    elif component_nom == 'O3':

        match_df.index.names = ['corner', 'ground_pixel', 'layer', 'level', 'scanline']
        
    match_df = match_df.groupby(by = ['layer', 'scanline', 'ground_pixel', 'time', 'delta_time']).mean()
    match_df = match_df.reset_index(level = ['layer', 'delta_time'])

    return match_df

In [ ]:
def TROPOMI_apply_avg_kernels(kernels_method, match_df, model_ds, TROPOMI_ds):

    """ Apply averaging kernels by using two methods:
        * Nearest neighbours: Find the nearest neighbours in the observation space (in pressures, latitude and longitudes)
        * Interpolation: Find the nearest neighbours in the observation space (in latitude and longitudes) and 
                         interpolate values in pressure

        Args:
            kernels_method (str): Method to apply averaging kernels to model space (Nearest neighbours or Interpolation)
            match_df (dataframe): Dataframe used to apply averaging kernels
            model_ds (xarray): Model levels dataset in xarray format
            TROPOMI_ds (xarray): TROPOMI dataset in xarray format
        
        Returns:
            match_df (dataframe): Dataframe used to apply averaging kernels
    """

    if kernels_method == 'Nearest neighbours':
        
        match_df = avg_kernels_nearest_neighbours(match_df, model_ds)

    elif kernels_method == 'Interpolation':
        
        print('The application of averaging kernels will start now.')
        match_df, match_df_visualize = avg_kernels_vertical_interpolation(match_df, model_ds, TROPOMI_ds)
        
        """
        answer = input('Do you want to see the vertical interpolation for one location? Yes or No:')
        
        if answer == 'Yes':
            scanline_value = input('Show interpolation for one scanline: ')
            ground_pixel_value = input('Show interpolation for one ground_pixel: ')
            visualize_interpolation(match_df_visualize, scanline_value, ground_pixel_value)
        
        else:
            pass
        """
    
    # Calculate values to generate CAMS column to sum in the next step
    if 'profile_apriori' in match_df.columns:
        match_df['model_column'] = match_df.apply(lambda row: row['profile_apriori'] +
                                                              row['kernel_column'] * row['model_component']  -
                                                              row['kernel_column'] * row['profile_apriori'], 
                                                              axis = 1)
    
    else:
        match_df['model_column'] = match_df.apply(lambda row: row['kernel_column'] * row['model_component'], 
                                                              axis = 1)

    return match_df

In [ ]:
def avg_kernels_nearest_neighbours(match_df, model_ds):

    """ Nearest neighbours method: Find the nearest neighbours in the observation space (in pressures, latitude and longitudes)

        Args:
            match_df (dataframe): Dataframe used to apply averaging kernels
            model_ds (xarray): Model levels dataset in xarray format
        
        Returns:
            match_df (dataframe): Dataframe used to apply averaging kernels
    """
    
    model_pressures = model_levels['ph [Pa]'].to_numpy()
    model_times = model_ds.valid_time.data

    match_df['lay_index'] = match_df.apply(lambda row: nearest_neighbour(model_pressures, row['pressure']), axis = 1)
    match_df['step_index'] = match_df.apply(lambda row: nearest_neighbour(model_times, row['delta_time']), axis = 1)
    match_df['model_time'] = match_df.apply(lambda row: model_ds.valid_time[row['step_index']].values, axis = 1)

    match_df['model_component'] = match_df.apply(lambda row: model_ds.component.sel( 
                                                            latitude = row['latitude'], 
                                                            longitude = row['longitude'], 
                                                            method = 'nearest').isel(hybrid = int(row['lay_index']), 
                                                            step = int(row['step_index'])).values, 
                                                            axis = 1)
    
    return match_df

In [ ]:
def avg_kernels_vertical_interpolation(match_df, model_ds, TROPOMI_ds):

    """ Interpolation: Find the nearest neighbours in the observation space (in latitude and longitudes) and 
        interpolate values in pressure

        Args:
            match_df (dataframe): Dataframe used to apply averaging kernels
            model_ds (xarray): Model levels dataset in xarray format
            TROPOMI_ds (xarray): TROPOMI dataset in xarray format
        
        Returns:
            match_df (dataframe): Dataframe used to apply averaging kernels
    """
    
    sensor_coords_df = TROPOMI_subset_coords(TROPOMI_ds)

    match_df.set_index('pressure', append=True, inplace=True)

    # Create index that includes CAMS pressure levels for all the locations in TROPOMI
    new_index = pd.MultiIndex.from_product([match_df.index.levels[0], 
                                            match_df.index.levels[1],
                                            match_df.index.levels[2],
                                            model_levels['ph [Pa]'].to_numpy()],
                                            names = ['scanline', 'ground_pixel', 'time', 'pressure'])
    
    # Append original and new indexes and reindex dataframe
    match_df = match_df.reindex(match_df.index.append(new_index))

    # Sort and reset index
    match_df = match_df.sort_index()
    match_df = match_df.reset_index()

    # Find latitudes in CAMS rows with scanlines and ground pixels
    match_df['latitude'] = match_df.apply(lambda row: float(sensor_coords_df[
                                                            (sensor_coords_df['scanline'] == row['scanline']) & 
                                                            (sensor_coords_df['ground_pixel'] == row['ground_pixel'])]['latitude'])
                                                            if pd.isnull(row['latitude']) else row['latitude'], 
                                                            axis = 1)
                                                            
    # Find longitudes in CAMS rows with scanlines and ground pixels
    match_df['longitude'] = match_df.apply(lambda row: float(sensor_coords_df[
                                                            (sensor_coords_df['scanline'] == row['scanline']) & 
                                                            (sensor_coords_df['ground_pixel'] == row['ground_pixel'])]['longitude'])
                                                            if pd.isnull(row['longitude']) else row['longitude'], 
                                                            axis = 1)
                                                            
    # Find hybrids in CAMS rows from 137 models table
    match_df['hybrid'] = match_df.apply(lambda row: nearest_neighbour(model_levels['ph [Pa]'].to_numpy(), row['pressure']) + 1
                                                    if pd.isnull(row[sensor_column]) else math.nan, 
                                                    axis = 1)

    # Get unique timestep
    first_delta_time = sensor_ds['delta_time'].isel(scanline = 0, time = 0).values
    unique_step = nearest_neighbour(model_ds.valid_time.data, first_delta_time)
    unique_time = model_ds.component.isel(step = unique_step).step.values.astype('timedelta64[h]')

    # Get CAMS component data at nearby TROPOMI locations (nearest neighbours)
    # Do it only for CAMS rows
    match_df['model_component'] = match_df.apply(lambda row: model_ds.component.sel(
                                                            hybrid = row['hybrid'], 
                                                            latitude = row['latitude'], 
                                                            longitude = row['longitude'], 
                                                            step = unique_time, method = 'nearest').values 
                                                            if pd.isnull(row[sensor_column]) else math.nan,
                                                            axis = 1)

    # Transform 1D-array data to float
    match_df['model_component'] = match_df['model_component'].apply(lambda x: float(x))

    # Set multiindex again and sort
    match_df = match_df.set_index(['time', 'pressure', 'scanline', 'ground_pixel'])
    match_df = match_df.sort_values(['time', 'ground_pixel','scanline', 'pressure'], 
                                    ascending = [True, True, True, False])

    # Interpolation
    match_df['model_component'] = match_df['model_component'].interpolate()

    # Show vertical interpolation for one location
    match_df_visualize = match_df

    # Drop unnecessary values
    match_df = match_df.drop(model_levels['ph [Pa]'].to_numpy(), level = 'pressure')
    
    model_times = model_ds.valid_time.data
    match_df['step_index'] = match_df.apply(lambda row: nearest_neighbour(model_times, row['delta_time']), axis = 1)

    # Reset pressure index
    match_df = match_df.reset_index('pressure')

    return match_df, match_df_visualize

In [ ]:
def visualize_interpolation(match_df_visualize, scanline_value, ground_pixel_value):

    """ Visualize interpolated partial columns of the model for a specific location given a scanline and ground pixel

        Args:
            match_df_visualize (dataframe): Dataframe used to apply averaging kernels with interpolated values
            scanline_value (int): Specific location scanline
            ground_pixel_value (int): Specific location ground pixel
    """

    # Query to get data for one location
    small = match_df.query('scanline == @scanline_value and ground_pixel == @ground_pixel_value')

    # Get pressure data
    all_pressures = small.index.get_level_values(1).to_numpy()
    model_pressures = model_levels['ph [Pa]'].to_numpy()

    # Show in black original values and in red the interpolated pressures
    diff_colors = np.where(np.isin(all_pressures, model_pressures), 'black', 
                        np.where(~np.isin(all_pressures, model_pressures), 'red', 'yellow'))

    # Show component vs. pressures
    plt.scatter(small['model_component'], all_pressures, c = diff_colors, s = 10)

    # Revert yaxis to have surface pressure on the bottom
    ax = plt.gca()
    ax.set_ylim(ax.get_ylim()[::-1])
    ax.set_xlabel(f'{component_nom} (molecules/cm²)', fontsize = 18)
    ax.set_ylabel('Pressure (Pa)', fontsize = 18)
    plt.show()